**TESTING THE EINSTEIN NOTATION IN SEIR2 SPATIAL - AGE-STRATIFIED MODEL**

In [75]:
# import dependencies

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# OPTIONAL: Load the "autoreload" extension so that package code can change
%load_ext autoreload
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
################
## Load model ##
################

from DENV_models_pySODM import JumpProcess_SEIR2_spatial_age_sto_einstein as DENV_model
print(DENV_model)
print(DENV_model.states)
print(DENV_model.parameters)

<class 'DENV_models_pySODM.JumpProcess_SEIR2_spatial_age_sto_einstein'>
['S', 'S1', 'S2', 'E1', 'E2', 'E12', 'E21', 'I1', 'I2', 'I12', 'I21', 'R1', 'R2', 'R', 'I_cum']
['alpha', 'beta_1', 'sigma', 'gamma', 'psi', 'ph', 'f_h', 'beta_0', 'ODmatrix', 'N']


In [77]:
#################
## Setup model ##
#################

# Define model parameters
p = 0
N_A = np.array([[10, 5],
                [5, 10]])
N_B = np.array([[10, 5],
                [5, 10]])
N_C = np.array([[10, 5],
                [5, 10]])
N = np.stack((N_A, N_B, N_C), axis=2) # turns N into 2x2x3 

OD_child = np.array([[0.6, 0.2, 0.20],
               [0.0, 1, 0.0],
               [0, 0, 1]])
OD_adult = np.array([[0.6, 0.2, 0.20],
               [0.0, 1, 0.0],
               [0, 0, 1]])
M = np.stack((OD_child, OD_adult), axis = 2) # turns M into 3x3x2

print("M shape",M.shape)
print("N shape",N.shape)


age_groups = pd.IntervalIndex.from_tuples([(0,15),(15,100)], closed='left')
locations = ['A', 'B', 'C']

params = {'alpha': 45 ,'sigma':5, 'gamma': 15, 'psi': 1.5, 'beta_0' : np.array([0.2, 0.10]), 'beta_1' : 0.5, 'ph':p, 'f_h':0.8, 'N': N, 'ODmatrix': M} # I made younger groups more infectious than older, beta_1 is the same for all age groups

# Define initial condition

initS = np.array([[80, 10, 1000],[10, 90, 4000]]) # age x location
initI1 = np.array([[10, 0, 0], [0, 0, 0]]) # age x location 

init_states = {'S': pd.DataFrame(initS, index=age_groups, columns=locations),
               'I1': pd.DataFrame(initI1, index=age_groups, columns=locations)}


# Define model coordinates
coordinates={'age_group': age_groups, 'location': locations}

M shape (3, 3, 2)
N shape (2, 2, 3)


In [78]:
####################
# Initialize model #
####################
model_SEIR2_Einstein = DENV_model(states=init_states, parameters=params, coordinates=coordinates) # no time_dependent_parameters

# THIS SEEMS TO BE WORKING :)

ODmatrix [[[0.6 0.6]
  [0.2 0.2]
  [0.2 0.2]]

 [[0.  0. ]
  [1.  1. ]
  [0.  0. ]]

 [[0.  0. ]
  [0.  0. ]
  [1.  1. ]]]
ODmatrix (3, 3, 2)
T_mob (2, 3)
I_frac shape (2, 3)
I_frac type <class 'numpy.ndarray'>
N shape (2, 2, 3)
I1 shape (2, 3)
I21 shape (2, 3)
T shape (2, 3)
test 1 il-> il for I_frac (2, 3)
test 2 il,ail->ail (2, 2, 3)
test 3 il,ail->al (2, 3)


index_M MultiIndex([(  [0, 15), 'A', 'A'),
            (  [0, 15), 'A', 'B'),
            (  [0, 15), 'A', 'C'),
            (  [0, 15), 'B', 'A'),
            (  [0, 15), 'B', 'B'),
            (  [0, 15), 'B', 'C'),
            (  [0, 15), 'C', 'A'),
            (  [0, 15), 'C', 'B'),
            (  [0, 15), 'C', 'C'),
            ([15, 100), 'A', 'A'),
            ([15, 100), 'A', 'B'),
            ([15, 100), 'A', 'C'),
            ([15, 100), 'B', 'A'),
            ([15, 100), 'B', 'B'),
            ([15, 100), 'B', 'C'),
            ([15, 100), 'C', 'A'),
            ([15, 100), 'C', 'B'),
            ([15, 100), 'C', 'C')],
           names=['age_group', 'location', 'location'])
index_N MultiIndex([(  [0, 15),   [0, 15)),
            (  [0, 15), [15, 100)),
            ([15, 100),   [0, 15)),
            ([15, 100), [15, 100))],
           names=['age_group', 'age_group'])
flattened_matrix [[0.6 0.2 0.2 0.  1.  0.  0.  0.  1. ]
 [0.6 0.2 0.2 0.  1.  0.  0.  0.  1. ]]
flattened_m